In [4]:
import numpy as np
import random
import pprint

In [5]:
class sys():
    def __init__(self, n_side):
        self.n_side = n_side
        self.K = 0.1 #J/kT
        self.lattice = np.zeros(shape=(n_side,n_side))
    
    def initialize(self, param):
        """
        Function to explicitly initialize the system in either an ordered or disordered state
        """
        if param.lower() == 'ordered':
            self.lattice = np.ones(shape=(self.n_side, self.n_side))
        
        elif param.lower() == 'disordered':
            for i in range(self.n_side):
                for j in range(self.n_side):
                    self.lattice[i][j] = np.random.randint(0,2) * 2 - 1 # +/- 1
        
        else:
            print("System must be initialized to either 'ordered' or 'disordered'")
            pass
        
        # Assign energy and total magnetization
        self.E, self.M = self.calculate_energy()
        
    def calculate_energy(self):
        """
        Calculate total energy (E) and magnetization (M) of the system.
        """
        E = sum(self.site_energy(i,j) for i in range(self.n_side) for j in range(self.n_side))
        M = sum(self.lattice[i][j] for i in range(self.n_side) for j in range(self.n_side))
        
        return E, M
    
    def site_energy(self, i, j):
        """
        Calculate energy of one site (spin) of the system. Used in calculate_energy function above.
        """
        left = (i - 1) % self.n_side
        right = (i + 1) % self.n_side
        up = (j + 1) % self.n_side
        down = (j - 1) % self.n_side
        neighbors = self.lattice[left][j] + self.lattice[right][j] + self.lattice[i][up] + self.lattice[i][down]
        
        en = -self.K * self.lattice[i][j] * neighbors
        return en

    

In [21]:
def metropolis(sys):
    """
    Implementation of the Metropolis-Hastings algorithm. Attempts to flip one spin at a time, chosen at random.
    """
    i = random.randrange(sys.n_side)
    j = random.randrange(sys.n_side)
    de = delta_e(sys, i, j)
    
    accept = False
    if de <= 0:
        accept = True
    else:
        if(random.random() < np.exp(-de)):
            accept = True
    
    if accept:
        sys.lattice[i][j] *= -1
        sys.E += de
        sys.M += 2 * sys.lattice[i][j]

def delta_e(sys, i, j):
    """
    Calculates the change in energy of a proposed spin flip. Used in metropolis function above.
    """
    de = -2 * sys.site_energy(i,j)
    return de

def trajectory(sys, steps, sample_freq):
    """
    Executes multiple iterations of metropolis algorithm and records the energy, magnetization, and total lattice.
    """
    traj = dict()
    count = 0
    for i in range(steps):
        metropolis(sys)
        if i % sample_freq == 0:
            sample = {'Energy': sys.E, 'Magnetization': sys.M, 'Lattice': sys.lattice}
            traj[count] = sample
            count += 1
    return traj
    

In [22]:
S = sys(n_side=5)
S.lattice

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [23]:
S.initialize('ordered')
S.lattice

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [24]:
print(S.E, S.M)

-10.000000000000004 25.0


In [25]:
traj = trajectory(S, 100, 10)
pprint.pprint(traj)

{0: {'Energy': -10.000000000000004,
     'Lattice': array([[ 1.,  1., -1., -1.,  1.],
       [-1.,  1., -1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1.],
       [ 1., -1., -1., -1.,  1.],
       [ 1.,  1., -1.,  1.,  1.]]),
     'Magnetization': 25.0},
 1: {'Energy': -8.400000000000002,
     'Lattice': array([[ 1.,  1., -1., -1.,  1.],
       [-1.,  1., -1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1.],
       [ 1., -1., -1., -1.,  1.],
       [ 1.,  1., -1.,  1.,  1.]]),
     'Magnetization': 21.0},
 2: {'Energy': -7.200000000000002,
     'Lattice': array([[ 1.,  1., -1., -1.,  1.],
       [-1.,  1., -1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1.],
       [ 1., -1., -1., -1.,  1.],
       [ 1.,  1., -1.,  1.,  1.]]),
     'Magnetization': 17.0},
 3: {'Energy': -6.000000000000002,
     'Lattice': array([[ 1.,  1., -1., -1.,  1.],
       [-1.,  1., -1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1.],
       [ 1., -1., -1., -1.,  1.],
       [ 1.,  1., -1.,  1.,  1.]]),
     'Magnetization'